### 1. L1正则化

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1
model = Sequential()
model.add(Dense(units=4, 
                activation='relu', 
                input_shape=(4,),
                kernel_regularizer=l1(0.01)))
model.add(Dense(units=1, 
                activation=None, 
                kernel_regularizer=l1(0.01)))

### 2. L2正则化

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2
model = Sequential()
model.add(Dense(units=4, 
                activation='relu', 
                input_shape=(4,),
                kernel_regularizer=l2(0.01)))
model.add(Dense(units=1, 
                activation=None, 
                kernel_regularizer=l2(0.01)))

### 3.增加训练集样本个数

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image_path = "cat.jpg"
# 加载图片
image = plt.imread(image_path)
# 打印图片
plt.imshow(image)
# 查看这张图片中每一个维度中元素个数
print(image.shape)

In [ ]:
generator = ImageDataGenerator(rotation_range=10, # 旋转
                               width_shift_range=0.1, # 宽度
                               height_shift_range=0.1, # 高度
                               shear_range=0.15, # 图片剪切
                               zoom_range=0.1, # 缩放
                               channel_shift_range=10, # 三通道转换
                               brightness_range=(0.9, 0.7), # 亮度
                               horizontal_flip=True) # 水平移动

In [ ]:
image = np.expand_dims(image, 0)
print(image.shape)

In [ ]:
aug_iter = generator.flow(image)
aug_images = [next(aug_iter)[0] for i in range(10)]

In [ ]:
def plots(ims, figsize=(12,6), rows=1):
    ims = np.array(ims).astype(np.uint8)
    fig = plt.figure(figsize=figsize)
    cols = len(ims) // rows if len(ims) % 2 == 0 else len(ims) // rows + 1
    for i in range(len(ims)):
        sp = fig.add_subplot(rows, cols, i + 1)
        sp.axis('Off')
        plt.imshow(ims[i])

In [ ]:
plots(aug_images, figsize=(20,7), rows=2)

### 4. Dropout的应用

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
# 第一个中间层应用Dropout
model.add(Dense(units=4, activation='relu', input_shape=(4,)))
model.add(Dropout(rate=0.5))
# 第二个中间层应用Dropout
model.add(Dense(units=4, activation='relu'))
model.add(Dropout(rate=0.5))
# 输出层
model.add(Dense(units=1, activation='relu'))
model.summary()

### 5. 早停法

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
# 加载鸢尾花数据集
iris = load_iris()
# 数据集中样本的特征值
X = iris.data
# 数据集中样本的标签值
y = iris.target
# 将数据集中的四分之三划分为训练集，四分之一划分为测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
# 分别将训练集与测试集样本标签进行独热编码处理
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(50, input_shape=(4,), activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_loss',
                        min_delta=1e-3,
                        patience=5,
                        mode='auto',
                        verbose=1,
                        restore_best_weights=True)

In [ ]:
model.fit(X_train, 
          y_train, 
          validation_data=(X_test, y_test),
          callbacks=[monitor],
          verbose=2,
          epochs=1000)